In [13]:
from ollama import chat
from rich.pretty import pprint as pp
from dotenv import load_dotenv
import os
load_dotenv("notebooks/.env")

True

In [14]:
print("OLLAMA_API_KEY exists:", bool(os.getenv("OLLAMA_API_KEY")))
print("OLLAMA_HOST:", os.getenv("OLLAMA_HOST"))
print("OLLAMA_BASE_URL:", os.getenv("OLLAMA_BASE_URL"))
ollama_base_url = os.getenv("OLLAMA_BASE_URL", os.getenv("OLLAMA_HOST"))
print("OLLAMA_BASE_URL:", ollama_base_url)
# ollama signout
# ollama signin

OLLAMA_API_KEY exists: True
OLLAMA_HOST: 0.0.0.0:11434
OLLAMA_BASE_URL: http://host.docker.internal:11434
OLLAMA_BASE_URL: http://host.docker.internal:11434


In [15]:
from tiny_agent.tools.decorator import tool

@tool()
def add(a:float, b:float) -> float:
    """Apply addition"""
    return a + b

@tool()
def sub(a:float, b:float) -> float:
    """Apply subtraction"""
    return a - b

@tool()
def mul(a:float, b:float) -> float:
    """Apply multiplication"""
    return a * b

@tool()
def div(a:float, b:float) -> float:
    """Apply division"""
    return a / b

In [16]:
from ollama import Client
from tiny_agent.agent.ollama_utils import ollama_automatic_function_calling

client = Client(host=ollama_base_url)

model = "glm-5:cloud"  # glm-5:cloud "glm-4.7-flash:bf16" "qwen3:1.7b"#  "qwen3:8b"
response, messages = ollama_automatic_function_calling(
    client.chat,
    messages=[
        {
            "role": "user",
            "content": "Die mathematische Frage: Wie viel ist 4 + 7 * 9 / 6?",
        }
    ],
    model=model,
    tools=[add, sub, mul, div],
    think="high",  # 'low', 'medium', 'high' or true, false
    options={
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
)

print("MODEL:", model)

MODEL: glm-5:cloud


In [17]:
pp(response)

ChatResponse(
│   model='glm-5:cloud',
│   created_at='2026-02-17T13:21:30.322295118Z',
│   done=True,
│   done_reason='stop',
│   total_duration=5426245807,
│   load_duration=None,
│   prompt_eval_count=697,
│   prompt_eval_duration=None,
│   eval_count=89,
│   eval_duration=None,
│   message=Message(
│   │   role='assistant',
│   │   content='Das Ergebnis ist **14,5**.\n\nDie Rechnung wird nach der Regel "Punkt-vor-Strich" durchgeführt:\n- Zuerst: 7 × 9 = 63\n- Dann: 63 ÷ 6 = 10,5\n- Zum Schluss: 4 + 10,5 = 14,5',
│   │   thinking='Perfect! The calculation is complete. The result is 14.5.',
│   │   images=None,
│   │   tool_name=None,
│   │   tool_calls=None
│   ),
│   logprobs=None
)

In [18]:
pp(messages)

[
│   {'role': 'user', 'content': 'Die mathematische Frage: Wie viel ist 4 + 7 * 9 / 6?'},
│   {
│   │   'role': 'assistant',
│   │   'content': 'Ich werde diese Rechnung Schritt für Schritt nach der Regel "Punkt vor Strich" (Multiplikation und Division vor Addition) lösen.\n\nZuerst berechne ich 7 * 9:',
│   │   'thinking': 'The user is asking a mathematical question in German: "Wie viel ist 4 + 7 * 9 / 6?" which translates to "How much is 4 + 7 * 9 / 6?"\n\nI need to solve this following the order of operations (PEMDAS/BODMAS):\n1. First, multiplication and division from left to right\n2. Then addition and subtraction from left to right\n\nSo the expression is: 4 + 7 * 9 / 6\n\nStep 1: 7 * 9 = 63\nStep 2: 63 / 6 = 10.5\nStep 3: 4 + 10.5 = 14.5\n\nLet me use the available tools to calculate this step by step.\n\nFirst, I\'ll multiply 7 * 9, then divide by 6, then add 4.',
│   │   'tool_calls': [{'function': {'name': 'mul', 'arguments': {'a': 7, 'b': 9}}}]
│   },
│   {'role': 'tool', 'content': '63', 'tool_name': 'mul'},
│   {
│   │   'role': 'assistant',
│   │   'content': '',
│   │   'thinking': 'Now I need to divide 63 by 6.',
│   │   'tool_calls': [{'function': {'name': 'div', 'arguments': {'a': 63, 'b': 6}}}]
│   },
│   {'role': 'tool', 'content': '10.5', 'tool_name': 'div'},
│   {
│   │   'role': 'assistant',
│   │   'content': '',
│   │   'thinking': 'Now I need to add 4 to 10.5.',
│   │   'tool_calls': [{'function': {'name': 'add', 'arguments': {'a': 4, 'b': 10.5}}}]
│   },
│   {'role': 'tool', 'content': '14.5', 'tool_name': 'add'},
│   {
│   │   'role': 'assistant',
│   │   'content': 'Das Ergebnis ist **14,5**.\n\nDie Rechnung wird nach der Regel "Punkt-vor-Strich" durchgeführt:\n- Zuerst: 7 × 9 = 63\n- Dann: 63 ÷ 6 = 10,5\n- Zum Schluss: 4 + 10,5 = 14,5',
│   │   'thinking': 'Perfect! The calculation is complete. The result is 14.5.'
│   }
]